# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
import math
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, mean_squared_error
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
import string
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import warnings
from keras import backend as K
warnings.filterwarnings("ignore")
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Data exploration

In [3]:
train_df = pd.read_csv('cleaned_train.csv', index_col='id')
train_df.head()

,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
id,,,,,,,,,,,,,,,,,,,,,
59848,0.000000,cool like would want mother read realli great ...,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
59849,0.000000,thank would make life lot less anxietyinduc ke...,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
59852,0.000000,urgent design problem kudo take impress,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
59855,0.000000,someth ill abl instal site releas,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
59856,0.893617,haha guy bunch loser,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [4]:
test_df = pd.read_csv('all_data.csv', index_col='id')
test_df.head()

,text
id,
1860808710928454067,thefrogsoupram bluerei naegiko didnt consent a...
1860798467011977633,kayninewrit rowanisr softagatha someon could w...
1860783005717176665,ai art revolut threat discuss ethic implic aig...
1860698588394971197,ai music video wake wildflow music video wake ...
1860694275467403362,skullxnft great concept honestli origin ethic ...


In [5]:
train_df.describe()

,target,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,...,parent_id,article_id,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
count,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,...,21000.000000,21000.000000,21000.000000,21000.000000,21000.0,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000
mean,0.084552,0.003952,0.014445,0.017568,0.065036,0.008954,0.001489,0.001174,0.000713,0.007602,...,133455.208857,58306.749238,0.000476,0.000048,0.0,1.618952,0.000048,0.006298,1.687714,8.535524
std,0.179662,0.021774,0.072130,0.069335,0.156771,0.048099,0.032361,0.031542,0.018030,0.079135,...,127320.131102,22846.173068,0.021817,0.006901,0.0,2.925190,0.006901,0.044362,31.246439,50.847671
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2006.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,4.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,42746.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,4.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,241888.000000,52124.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,4.000000
75%,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255676.750000,55785.000000,0.000000,0.000000,0.0,2.000000,0.000000,0.000000,0.000000,6.000000
max,1.000000,0.300000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,267814.000000,106343.000000,1.000000,1.000000,0.0,60.000000,1.000000,0.948495,1842.000000,1867.000000


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21000 entries, 59848 to 267822
Data columns (total 44 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   target                               21000 non-null  float64
 1   comment_text                         20950 non-null  object 
 2   severe_toxicity                      21000 non-null  float64
 3   obscene                              21000 non-null  float64
 4   identity_attack                      21000 non-null  float64
 5   insult                               21000 non-null  float64
 6   threat                               21000 non-null  float64
 7   asian                                21000 non-null  float64
 8   atheist                              21000 non-null  float64
 9   bisexual                             21000 non-null  float64
 10  black                                21000 non-null  float64
 11  buddhist                    

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 343 entries, 1860808710928454067 to 1859715514517553397
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    342 non-null    object
dtypes: object(1)
memory usage: 5.4+ KB


In [8]:
train_df.isnull().sum()


target                                  0
comment_text                           50
severe_toxicity                         0
obscene                                 0
identity_attack                         0
insult                                  0
threat                                  0
asian                                   0
atheist                                 0
bisexual                                0
black                                   0
buddhist                                0
christian                               0
female                                  0
heterosexual                            0
hindu                                   0
homosexual_gay_or_lesbian               0
intellectual_or_learning_disability     0
jewish                                  0
latino                                  0
male                                    0
muslim                                  0
other_disability                        0
other_gender                      

In [9]:
test_df.isnull().sum()

text    1
dtype: int64

In [10]:
train_df.dropna(inplace=True)
train_df.isnull().sum()

target                                 0
comment_text                           0
severe_toxicity                        0
obscene                                0
identity_attack                        0
insult                                 0
threat                                 0
asian                                  0
atheist                                0
bisexual                               0
black                                  0
buddhist                               0
christian                              0
female                                 0
heterosexual                           0
hindu                                  0
homosexual_gay_or_lesbian              0
intellectual_or_learning_disability    0
jewish                                 0
latino                                 0
male                                   0
muslim                                 0
other_disability                       0
other_gender                           0
other_race_or_et

In [11]:
test_df.dropna(inplace=True)
test_df.isnull().sum()

text    0
dtype: int64

In [12]:
print("Train and test shape: {} {}".format(train_df.shape, test_df.shape))

Train and test shape: (20950, 44) (342, 1)


# Model

In [20]:
feature = train_df[['comment_text']]
output = train_df[['target']] 

X_train, X_val, y_train, y_val = train_test_split(feature, output, test_size=0.25, random_state=5400)

In [21]:
# Configuration parameters
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 1e-5
MAX_LEN = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## BERT embedding generation and dataset definition

In [22]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class ToxicityDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "targets": torch.tensor(target, dtype=torch.float),
        }

# data loader
def create_data_loader(df, targets, tokenizer, max_len, batch_size):
    dataset = ToxicityDataset(
        texts=df['comment_text'].values,
        targets=targets.values.flatten(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_loader = create_data_loader(X_train, y_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_loader = create_data_loader(X_val, y_val, tokenizer, MAX_LEN, BATCH_SIZE)


## BERT Regression Model Definition

In [23]:
class BertRegressionModel(nn.Module):
    def __init__(self, bert_model_name, output_size):
        super(BertRegressionModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.drop = nn.Dropout(0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, output_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.drop(pooled_output)
        return self.out(output)

model = BertRegressionModel("bert-base-uncased", output_size=1).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.MSELoss()


In [ ]:
from tqdm import tqdm

# train
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss/len(train_loader):.4f}")


Epoch 1: 100%|██████████| 131/131 [05:05<00:00,  2.33s/it, loss=0.0715] 


Epoch 1/3, Loss: 0.0556


Epoch 2: 100%|██████████| 131/131 [05:00<00:00,  2.29s/it, loss=0.0641]


Epoch 2/3, Loss: 0.0416


Epoch 3: 100%|██████████| 131/131 [05:03<00:00,  2.31s/it, loss=0.0394] 

Epoch 3/3, Loss: 0.0340


## Obscene

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

print("Processing target: obscene")

# Update feature and output
feature = train_df[["comment_text"]]
output = train_df[["obscene"]]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(feature, output, test_size=0.25, random_state=5400)

# Create data loaders
train_loader = create_data_loader(X_train, y_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_loader = create_data_loader(X_val, y_val, tokenizer, MAX_LEN, BATCH_SIZE)

# Train the model
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)  # Progress bar
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Update progress bar
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{EPOCHS}, Average Loss: {train_loss / len(train_loader):.4f}")




Processing target: obscene


Epoch 1: 100%|██████████| 66/66 [04:38<00:00,  4.23s/it, loss=0.00152]


Epoch 1/3, Average Loss: 0.0085


Epoch 2: 100%|██████████| 66/66 [02:49<00:00,  2.57s/it, loss=0.00935]


Epoch 2/3, Average Loss: 0.0069


Epoch 3: 100%|██████████| 66/66 [02:30<00:00,  2.28s/it, loss=0.00351]

Epoch 3/3, Average Loss: 0.0064


In [ ]:
# Evaluate on validation set
model.eval()
val_loss = 0
with torch.no_grad():
    for data in val_loader:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        val_loss += loss.item()

print(f"Validation Loss: {val_loss / len(val_loader):.4f}")


In [ ]:

# Predict on test set
test_loader = create_data_loader(test_df, pd.DataFrame([0] * len(test_df), columns=["dummy_target"]), tokenizer, MAX_LEN, BATCH_SIZE)

predictions = []
print("Starting predictions...")
with torch.no_grad():
    loop = tqdm(test_loader, leave=True)  # Progress bar for prediction
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())


In [ ]:

# Save predictions
test_df["predicted_obscene"] = predictions
print("Predictions for 'obscene' added to test_df.")

# Save to CSV
output_path = "data/data-predict/BERT_test_predictions_obscene.csv"
test_df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}.")

## identity_attack


In [31]:

print("Processing target: identity_attack")

# Update feature and output
feature = train_df[["comment_text"]]
output = train_df[["identity_attack"]]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(feature, output, test_size=0.25, random_state=5400)

# Create data loaders
train_loader = create_data_loader(X_train, y_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_loader = create_data_loader(X_val, y_val, tokenizer, MAX_LEN, BATCH_SIZE)

# Train the model
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)  # Progress bar
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Update progress bar
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{EPOCHS}, Average Loss: {train_loss / len(train_loader):.4f}")




Processing target: identity_attack


Epoch 1:   0%|          | 2/982 [00:06<56:44,  3.47s/it, loss=0.00703]


KeyboardInterrupt: 

In [ ]:
# Evaluate on validation set
model.eval()
val_loss = 0
with torch.no_grad():
    for data in val_loader:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        val_loss += loss.item()

print(f"Validation Loss: {val_loss / len(val_loader):.4f}")


In [ ]:

# Predict on test set
test_loader = create_data_loader(test_df, pd.DataFrame([0] * len(test_df), columns=["dummy_target"]), tokenizer, MAX_LEN, BATCH_SIZE)

predictions = []
print("Starting predictions...")
with torch.no_grad():
    loop = tqdm(test_loader, leave=True)  # Progress bar for prediction
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())


In [ ]:
# Save predictions
test_df["predicted_obscene"] = predictions
print("Predictions for 'obscene' added to test_df.")

# Save to CSV
output_path = "data/data-predict/BERT_test_predictions_identity_attack.csv"
test_df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}.")

## insult

In [ ]:

print("Processing target: insult")

# Update feature and output
feature = train_df[["comment_text"]]
output = train_df[["insult"]]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(feature, output, test_size=0.25, random_state=5400)

# Create data loaders
train_loader = create_data_loader(X_train, y_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_loader = create_data_loader(X_val, y_val, tokenizer, MAX_LEN, BATCH_SIZE)

# Train the model
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)  # Progress bar
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Update progress bar
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{EPOCHS}, Average Loss: {train_loss / len(train_loader):.4f}")




In [ ]:
# Evaluate on validation set
model.eval()
val_loss = 0
with torch.no_grad():
    for data in val_loader:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        val_loss += loss.item()

print(f"Validation Loss: {val_loss / len(val_loader):.4f}")


In [ ]:

# Predict on test set
test_loader = create_data_loader(test_df, pd.DataFrame([0] * len(test_df), columns=["dummy_target"]), tokenizer, MAX_LEN, BATCH_SIZE)

predictions = []
print("Starting predictions...")
with torch.no_grad():
    loop = tqdm(test_loader, leave=True)  # Progress bar for prediction
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())


In [ ]:
# Save predictions
test_df["predicted_obscene"] = predictions
print("Predictions for 'obscene' added to test_df.")

# Save to CSV
output_path = "data/data-predict/BERT_test_predictions_insult.csv"
test_df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}.")

## threat

In [ ]:

print("Processing target: threat")

# Update feature and output
feature = train_df[["comment_text"]]
output = train_df[["threat"]]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(feature, output, test_size=0.25, random_state=5400)

# Create data loaders
train_loader = create_data_loader(X_train, y_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_loader = create_data_loader(X_val, y_val, tokenizer, MAX_LEN, BATCH_SIZE)

# Train the model
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)  # Progress bar
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Update progress bar
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{EPOCHS}, Average Loss: {train_loss / len(train_loader):.4f}")




In [ ]:
# Evaluate on validation set
model.eval()
val_loss = 0
with torch.no_grad():
    for data in val_loader:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        targets = data["targets"].to(DEVICE).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        val_loss += loss.item()

print(f"Validation Loss: {val_loss / len(val_loader):.4f}")


In [ ]:

# Predict on test set
test_loader = create_data_loader(test_df, pd.DataFrame([0] * len(test_df), columns=["dummy_target"]), tokenizer, MAX_LEN, BATCH_SIZE)

predictions = []
print("Starting predictions...")
with torch.no_grad():
    loop = tqdm(test_loader, leave=True)  # Progress bar for prediction
    for data in loop:
        input_ids = data["input_ids"].to(DEVICE)
        attention_mask = data["attention_mask"].to(DEVICE)
        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())


In [ ]:
# Save predictions
test_df["predicted_obscene"] = predictions
print("Predictions for 'obscene' added to test_df.")

# Save to CSV
output_path = "data/data-predict/BERT_test_predictions_insult.csv"
test_df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}.")